Thank you @ehekatlact for sharing the idea of using the na count of each record, and @hiro5299834 and @edrickkesuma for the great improvements, please upvote the following notebooks:
* https://www.kaggle.com/code/ehekatlact/tps2206-the-na-count-of-each-record-is-critical
* https://www.kaggle.com/code/hiro5299834/tps-jun-2022-pytorch-lightning-with-na-counts
* https://www.kaggle.com/code/edrickkesuma/np-random-top-public-notebook

Since the notebooks above were made using PyTorch, I thought it would be very interesting and fun to create a TensorFlow implementation of this idea. The only modifications I made besides the translation to TensorFlow were:
* Using weight normalization in the dense layers.
* Pre-calculating a matrix of negative ones whose rows are randomly shuffled through "tf.keras.utils.Sequence" at each epoch.

I tried to make it as simple as possible. I hope you find it as interesting as I did. Thank you for reading!

In [1]:
import os, random
from datetime import datetime
import numpy as np, pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Tensorflow:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow.keras import layers as L
from tensorflow_addons.activations import mish
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint

RANDOM_STATE=1967

C:\Users\orovi\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
# Input:
data = pd.read_csv('./input/data.csv',index_col=0)
sample_submission = pd.read_csv('./input/sample_submission.csv',index_col=0)

# Column groups:
f1col = [x for x in data.columns if x.startswith('F_1')]
f2col = [x for x in data.columns if x.startswith('F_2')]
f3col = [x for x in data.columns if x.startswith('F_3')]
f4col = [x for x in data.columns if x.startswith('F_4')]

# Mean imputation for F1 and F3 columns:
data[f1col+f3col] = data[f1col+f3col].fillna(data[f1col+f3col].mean())

# Minus one imputation of F4 columns in the testing set:
data_test = data.copy()
data_test[f4col] = data_test[f4col].fillna(-1)

# Row-wise NaN counts of F4 columns:
data[f4col].isna().sum(axis=1).value_counts()

0    759268
1    211342
2     27127
3      2124
4       135
5         4
dtype: int64

In [3]:
def get_model(hidden_size=64):
    
    # Input:
    inputF4 = L.Input(shape=len(f4col)-1)

    # Network:
    x = L.Dense(units=hidden_size*4,  
                kernel_regularizer=tf.keras.regularizers.l2(40e-6),
                activation=mish)(inputF4)
#    x = L.BatchNormalization()(x)
    x = L.Dense(units=hidden_size*4, 
                kernel_regularizer=tf.keras.regularizers.l2(40e-6),
                activation=mish)(x)
#    x = L.BatchNormalization()(x)
    x = L.Dense(units=hidden_size*2,
                kernel_regularizer=tf.keras.regularizers.l2(40e-6),
                activation=mish)(x)
#    x = L.BatchNormalization()(x)
    x = L.Dense(units=hidden_size, 
                kernel_regularizer=tf.keras.regularizers.l2(40e-6),
                activation=mish)(x)
#    x = L.BatchNormalization()(x)
    x = L.Dense(units=1, activation='linear')(x)
    
    # Output:
    model = tf.keras.Model(inputF4, x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    return(model)

class CustomDataset(tf.keras.utils.Sequence):
    
    def __init__(self, X, y, noise, batch_size=256, random_state=42):
        '''initialize dataset and noise positions'''
        np.random.seed(random_state)
        tf.random.set_seed(random_state)
        self.X, self.y, self.noise = X, y, noise
        self.batch_size = batch_size
        self.on_epoch_end()
        
    def __len__(self):
        '''number of batches per epoch'''
        return self.X.shape[0] // self.batch_size
    
    def __getitem__(self, index):
        '''generate batch and Randomly add -1 to the training and validation sets
        in the same amount as expected in the testing set'''
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_noises = self.noiseindexes[index*self.batch_size:(index+1)*self.batch_size]
        X_batch, y_batch = self.X[batch_indexes], self.y[batch_indexes]
        for i in range(self.noise.shape[1]):
            X_batch[range(self.batch_size),self.noise[batch_noises,i]] = -1  
        return X_batch, y_batch
    
    def on_epoch_end(self):
        '''shuffle the training set as well as the -1 positions'''
        self.indexes = np.arange(len(self.X))
        self.noiseindexes = np.arange(len(self.noise))
        np.random.shuffle(self.indexes)
        np.random.shuffle(self.noiseindexes)

Change the line below to "True" to create the models instead of loading from the dataset. I created the models in my personal computer and uploaded them afterwards since I almost run out of GPU time.

In [4]:
train = True

In [5]:
result = data.copy()

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
os.environ['PYTHONHASHSEED'] = str(RANDOM_STATE)
    
for cnt in range(5):
    
    # Prepare values which will be set to -1 during training:
    noise = np.array([np.random.choice(
        len(f4col)-1, size=cnt, replace=False
    ) for _ in range(data.shape[0])])
    
    # Find rows with zero-nan and with "cnt+1" nans:
    rownan0 = data[f4col].isna().sum(axis=1)==0
    rownans = data[f4col].isna().sum(axis=1)==cnt+1
    
    for col in f4col:
        
        print(f'***** {datetime.now().strftime("%H:%M:%S")} FITTING {col}-{cnt}')
        # Get testing set from rows with the amount of nan given by the current 'cnt' value:
        colna = data[col].isna()
        X_test = data_test.loc[rownans&colna,f4col].drop(columns=col).values
        print("x_test shape:", X_test.shape)
        
        # If there are NaNs in the testing set:
        if(X_test.shape[0]):
        
            # Get training and validation sets from zero-nan rows:
            X = data.loc[rownan0,f4col]
            y = X.pop(col)
            X_train, X_valid, y_train, y_valid, noise_train, noise_valid = train_test_split(
                X.values, y.values, noise[rownan0], train_size=0.8, random_state=cnt
            )

            # Training:
            model = get_model()
            if train:
                history = model.fit(
                    CustomDataset(X_train, y_train, noise_train), 
                    validation_data = CustomDataset(X_valid, y_valid, noise_valid),
                    epochs=300, 
                    verbose=0,
                    #use_multiprocessing=True, workers=4, 
                    callbacks=[
                        ReduceLROnPlateau(monitor='val_loss',mode='min',
                            verbose=0,factor=0.5,patience=3),
                        EarlyStopping(mode='min',restore_best_weights=True,
                            verbose=0,min_delta=1e-4,patience=10),
                        ModelCheckpoint(f'model_{cnt}_{col}.hdf5',monitor='val_loss',mode='min',
                            verbose=0,save_best_only=True,save_weights_only=True),
                    ]
                )
                model.load_weights(f'model_{cnt}_{col}.hdf5')
            else:
                model.load_weights(f'model_{cnt}_{col}.hdf5')

            # Performance:
            for i in range(cnt):
                X_valid[range(len(X_valid)),noise_valid[:,i]] = -1  
            y_pred = model.predict(X_valid)
            print(f'FFNN: cnt={cnt+1}, col={col}, RMSE={mean_squared_error(y_valid,y_pred,squared=False)}')        

            # Inference:
            result.loc[rownans&colna,col] = model.predict(X_test)
            K.clear_session()

***** 07:12:15 FITTING F_4_0-0
x_test shape: (14175, 14)
FFNN: cnt=1, col=F_4_0, RMSE=0.011026984679419075
***** 07:24:33 FITTING F_4_1-0
x_test shape: (14136, 14)
FFNN: cnt=1, col=F_4_1, RMSE=0.00973634833254945
***** 07:33:33 FITTING F_4_2-0
x_test shape: (14248, 14)
FFNN: cnt=1, col=F_4_2, RMSE=0.008351973182809185
***** 07:40:43 FITTING F_4_3-0
x_test shape: (13968, 14)
FFNN: cnt=1, col=F_4_3, RMSE=0.03490054361915114
***** 07:44:20 FITTING F_4_4-0
x_test shape: (13963, 14)
FFNN: cnt=1, col=F_4_4, RMSE=0.008366267836842755
***** 07:51:30 FITTING F_4_5-0
x_test shape: (14030, 14)
FFNN: cnt=1, col=F_4_5, RMSE=0.008576107699857387
***** 07:59:56 FITTING F_4_6-0
x_test shape: (14201, 14)
FFNN: cnt=1, col=F_4_6, RMSE=0.013529743386085409
***** 08:12:43 FITTING F_4_7-0
x_test shape: (13994, 14)
FFNN: cnt=1, col=F_4_7, RMSE=0.007745134700627013
***** 08:20:57 FITTING F_4_8-0
x_test shape: (13971, 14)
FFNN: cnt=1, col=F_4_8, RMSE=0.010057840820294825
***** 08:26:05 FITTING F_4_9-0
x_test s

In [6]:
# Submission:
for i in sample_submission.index:
    row, col = i.split('-')
    sample_submission.loc[i,'value'] = result.loc[int(row),col]
    
sample_submission.to_csv('submission.csv')
sample_submission.head()

,value
row-col,
0-F_1_14,-0.000905
0-F_3_23,0.000365
1-F_3_24,-0.000817
2-F_1_2,0.000551
2-F_4_2,0.377539


Thank you for reading! Please let me know if you have any questions or suggestions.